In [10]:
%pwd

'E:\\DeepLearning\\Kidney-Disease-Classification'

In [11]:
import os
import sys
from pathlib import Path

import os
from pathlib import Path

# Set your working directory explicitly
project_dir = Path(r"E:\DeepLearning\Kidney-Disease-Classification")
os.chdir(project_dir)
print("Working directory set to:", os.getcwd())


Working directory set to: E:\DeepLearning\Kidney-Disease-Classification


In [12]:
import tensorflow as tf
from pathlib import Path

# Enable eager execution (must come before model loading/training)
tf.config.run_functions_eagerly(True)
print("Eager Execution Enabled:", tf.executing_eagerly())



Eager Execution Enabled: True


In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf


In [2]:
from pathlib import Path

project_dir = Path(r"E:\DeepLearning\Kidney-Disease-Classification")
ROOT_DIR = Path.cwd()
CONFIG_FILE_PATH = ROOT_DIR / "config" / "config.yaml"
PARAMS_FILE_PATH = ROOT_DIR / "params.yaml"

In [4]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories
import tensorflow as tf

In [11]:
# NOTE: Please ensure that the cell defining TrainingConfig (cell 3) has been executed before running this cell.

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )
        return training_config


NameError: name 'TrainingConfig' is not defined